# Задание 1

Есть следующий код для [автоматического дифференцирования](https://en.wikipedia.org/wiki/Automatic_differentiation), в котором используются особенности системы типов языка `Python`: 

In [2]:
from scipy.misc import derivative
import numpy as np

In [15]:
from dataclasses import dataclass
from typing import Union, Callable
from numbers import Number

@dataclass
class Dual:
    value: float
    d: float

    def __add__(self, other: Union["Dual", Number]) -> "Dual":
        match other:
            case Dual(o_value, o_d):
                return Dual(self.value + o_value, self.d + o_d)
            case Number():
                return Dual(float(other) + self.value, self.d)
    
    #функция вычетания
    def __sub__(self, other: Union["Dual", Number]) -> "Dual":
        match other:
            case Dual(o_value, o_d):
                return Dual(self.value - o_value, self.d - o_d)
            case Number():
                return Dual(self.value - float(other), self.d)

    def __rsub__(self, other) -> "Dual":
        return -self.__sub__(other)

    def __mul__(self, other: Union["Dual", Number]) -> "Dual":
        match other:
            case Dual(o_value, o_d):
                return Dual(self.value * o_value, self.value * o_d + self.d * o_value)
            case Number():
                return Dual(float(other) * self.value, float(other) * self.d)    
    
    #унарные функции
    def __neg__(self) -> "Dual":                #?
        return Dual(-1 * self.value, -1*self.d)

    def __pos__(self) -> "Dual":                #?
        return Dual(self.value, self.d)

    def __abs__(self) -> "Dual":                
            return Dual(np.abs(self.value), self.d * self.value / self.value)
            
    def __invert__(self) -> "Dual":
            return Dual(self.value * (-1), self.d * (-1))
    __rmul__ = __mul__  # https://docs.python.org/3/reference/datamodel.html#object.__mul__
    __radd__ = __add__  # https://docs.python.org/3/reference/datamodel.html#object.__radd__
    
    #деление
    def __truediv__(self, other: Union["Dual", Number]) -> "Dual":
        match other:
            case Dual(o_value, o_d):
                if o_value == 0:
                    raise ZeroDivisionError
                else:
                    return Dual(self.value/o_value,  (self.d * o_value - self.value * o_d)/(o_value * o_value))
            case Number():
                if other == 0:
                    raise ZeroDivisionError
                else:
                    return self.__mul__(1/other)
    
    def __rtruediv__(self, other: Union["Dual", Number]) -> "Dual":
        match other:
            case Dual(o_value, o_d):
                if self == 0:
                    raise ZeroDivisionError
                else:
                    return other.__truediv__(self)
            case Number():
                if self == 0:
                    raise ZeroDivisionError
                else:
                    return Dual(1,d=self.d)/self.__truediv__(other)
    
    #возведение в степень
    def __pow__(self, other) -> "Dual":
        assert isinstance(other, (int, float)), "only support for int/float powers"
        if other == 0:
            return Dual(1, 1)
        elif other == 1: return Dual(self.value, self.d)
        else:    
            return Dual(pow(self.value, other) , other * self.d * pow(self.value, other - 1))
    
    def exp(self):
        return Dual(np.exp(self.value), self.d*np.exp(self.value))

    def sin(self):
        return Dual(np.sin(self.value), self.d * np.cos(self.value))

    def cos(self): 
        return Dual(np.cos(self.value), -1*self.d * np.sin(self.value))

    def log(self):
        return Dual(np.log(self.value), 1/(self.value)*self.d)

    def diff(func: Callable[[float], float]) -> Callable[[float], float]:
        return lambda x: func(Dual(x, 1.0)).d

    #дифференцирование с 2-мя переменными
    def diff_2V(func: Callable[[float], float]) -> Callable[[float], float]:
        return lambda x, y: [func(Dual(x, 1.0), Dual(y, 0.0)).d, func(Dual(x, 0.0), Dual(y, 1.0)).d]
    
    #дифференцирование с 3-мя переменными
    def diff_3V(func: Callable[[float], float]) -> Callable[[float], float]:
        return lambda x, y, z: [func(Dual(x, 1.0), Dual(y, 0.0), Dual(z, 0.0)).d, func(Dual(x, 0.0), Dual(y, 1.0), Dual(z, 0.0)).d, func(Dual(x, 0.0), Dual(y, 0.0), Dual(z, 01.0)).d]

    

Поддерживаются две операции - сложение и умножение. Применить можно так:

In [112]:
# Функция, которую будем дифференцировать
def f(x: float) -> float:
    return 5 * x * x + 2 * x + 2

f_diff = Dual.diff(f)

# значение производной в точке x = 2
f_diff(2)

22.0

## Задание 1.1 (5 баллов)

Какие недостатки вы видите в данной реализации? Реализуйте поддержку (полностью самостоятельно или модифицируя приведенный код):
- [унарных операций](https://docs.python.org/3/reference/datamodel.html#object.__neg__) 
- деления
- возведения в степень

Каким образом можно проверить корректность решения?  Реализуйте достаточный, по вашему мнению, набор тестов.

In [114]:
# ваш код
#Недостатки: поддержка конструкции match...case начиная с версии python 3.10

#Функции для тестирования

def f1(x: float) -> float:
    return +-+-+x

def f2(x: float) -> float:
    return abs(x)

def f3(x: float) -> float:
    return np.invert((-2)*x)

def f4(x: float) -> float:
    return x/8 + x/(x+3) + x**2


diff_f1 = Dual.diff(f1)
diff_f2 = Dual.diff(f2)
diff_f3 = Dual.diff(f3)
diff_f4 = Dual.diff(f4)

#сравнить с расчетами готовых пакетов
print("pos and neg: ", diff_f1(4))
print(derivative(f1,4.))
print("Abs: ", diff_f2(2))
print(derivative(f2,2.))
print("Invert: ", diff_f3(-1))
#print(derivative(f3,-1.))

print("truediv and pow: ", diff_f4(2))
#print(derivative(f4,2.))
print(derivative(f4, 2.))

pos and neg:  1.0
1.0
Abs:  1.0
1.0
Invert:  2.0
truediv and pow:  4.245
4.25


## Задание 1.2 (7 баллов)
Придумайте способ и реализуйте поддержку функций:
- `exp()`
- `cos()`
- `sin()`
- `log()`

Добавьте соответствующие тесты

In [13]:
# ваш код

#тестирование
def f5(x: float) -> float:
    return np.sin(x) + 5*np.cos(x) 

def f6(x: float) -> float:
    return np.log(x**2) 

def f7(x: float) -> float:
    return np.exp(x) 

diff_f5 = Dual.diff(f5)
diff_f6 = Dual.diff(f6)
diff_f7 = Dual.diff(f7)

print("sin and cos: ", diff_f5(10))
print(derivative(f5, 10.))

print("log: ", diff_f6(10))
print(derivative(f6, 10.))

print("exp: ", diff_f7(0.5))
print(derivative(f7, 0.5))


sin and cos:  1.881034025370396
1.5828355537855894
log:  0.2
0.2006706954621511
exp:  1.6487212707001282
1.9375792053127157


## Задание 1.3 (3 балла)

Воспользуйтесь методами **численного** дифференцирования для "проверки" работы кода на нескольких примерах. Например,  библиотеке `scipy` есть функция `derivative`. Или реализуйте какой-нибудь метод численного дифференцирования самостоятельно (**+5 баллов**)

In [117]:

def f8(x: float) -> float:
    return 5 * x * x + x/5 + 2 + np.cos(x)

def f9(x: float) -> float:
    return 6 + np.log(pow(x,3)) - 2*x/(x + pow(x,2))

def f10(x: float) -> float:
    return x/(x**2-x)

def f11(x: float) -> float:
    return np.sin(x) + np.cos(x)

diff_f8 = Dual.diff(f8)
diff_f9 = Dual.diff(f9)
diff_f10 = Dual.diff(f10)
diff_f11 = Dual.diff(f11)

print("AD result = ", diff_f8(5.), ", scipy's result = ", derivative(f8, 5.))
print("AD result = ", diff_f9(10.), ", scipy's result = ", derivative(f9, 10.))
print("AD result = ", diff_f10(5), ", scipy's result = ", derivative(f10, 5.))
print("AD result = ", diff_f11(0.5), ", scipy's result = ", derivative(f11, 0.5))

AD result =  51.15892427466314 , scipy's result =  51.00690695375699
AD result =  0.3165289256198347 , scipy's result =  0.3176727098598935
AD result =  -0.0625 , scipy's result =  -0.06666666666666665
AD result =  0.39815702328616975 , scipy's result =  0.3350375824927938


In [6]:
# ваш код

## Задание 1.4 (10 баллов)

Необходимо разработать систему автоматического тестирования алгоритма дифференцирования в следующем виде:
- реализовать механизм генерации "случайных функций" (например, что-то вроде такого: $f(x) = x + 5 * x - \cos(20 * \log(12 - 20 * x * x )) - 20 * x$ )
- сгенерировать достаточно большое число функций и сравнить результаты символьного и численного дифференцирования в случайных точках 

Генерацию случайных функций можно осуществить, например, двумя путями. 
1. Генерировать функцию в текстовом виде, зачем использовать встроенную функцию [eval](https://docs.python.org/3/library/functions.html#eval)

```python
func = eval("lambda x: 2 * x + 5")
assert func(42) == 89 
```

2. Использовать стандартный модуль [ast](https://docs.python.org/3/library/ast.html), который позволяет во время выполнения программы манипулировать [Абстрактным Синтаксическим Деревом](https://ru.wikipedia.org/wiki/%D0%90%D0%B1%D1%81%D1%82%D1%80%D0%B0%D0%BA%D1%82%D0%BD%D0%BE%D0%B5_%D1%81%D0%B8%D0%BD%D1%82%D0%B0%D0%BA%D1%81%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%BE%D0%B5_%D0%B4%D0%B5%D1%80%D0%B5%D0%B2%D0%BE).
Например, выражение 

```python
func = lambda x: 2 * x + 5
```

Можно запрограммировать с помощью кода:

```python

expr = ast.Expression(
    body=ast.Lambda(
        args=ast.arguments(
            args=[
                ast.arg(arg='x')
            ],
            posonlyargs=[],
            kwonlyargs=[],
            kw_defaults=[],
            defaults=[]
        ),
        body=ast.BinOp(
            left=ast.BinOp(
                left=ast.Constant(value=2),
                op=ast.Mult(),
                right=ast.Name(id='x', ctx=ast.Load())
            ),
            op=ast.Add(),
            right=ast.Constant(value=5)
        )
    )
)

ast.fix_missing_locations(expr)

func = eval(compile(expr, filename="", mode="eval"))

assert func(42) == 89
```

При реализации нужно учитывать области допустимых значений функций.

In [ ]:
# ваш код

## Задание 1.5 (7 баллов)

Реализуйте поддержку функций нескольких аргументов. Например

```python
def f(x: float, y: float, z: float) -> float:
    return x * y + z - 5 * y  


f_diff = diff(f)

f_diff(10, 10, 10) # = [10, 5, 1]
```

In [18]:
# ваш код

#тестирование для нескольких переменных
def f_2V(x: float, y: float) -> float:
    return 3*x*x - 2*y*y*y

def f_3V(x: float, y: float, z: float) -> float:
    return x * y + z - 5 * y  

f_2V_diff = Dual.diff_2V(f_2V)
f_3V_diff = Dual.diff_3V(f_3V)
x = 10
y = 10 
z = 10
print(f_2V_diff(1.5, 3.28)) #= [9, -64.5504] (пример заранее рассчитан)
f_3V_diff(x, y, z)

[9.0, -64.5504]


[10.0, 5.0, 1.0]